In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_curve, auc
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, SpatialDropout1D, Conv1D, MaxPooling1D
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, GlobalMaxPooling1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers.legacy import Adam 
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from transformers import BertTokenizer, TFBertModel
import nltk
# Download necessary NLTK resources
#NLTK (Natural Language Toolkit) is a Python library for text processing tasks like tokenization, part-of-speech tagging, and sentiment analysis.
try:
    nltk.data.find('tokenizers/punkt')
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('punkt')
    nltk.download('wordnet')

def load_farm_ads_data(text_file: str, vector_file: str):
    # Dictionary to store index:value pairs
    text_data = []
    # Reads the data and separates the label and text
    with open(text_file, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split()
            if not parts:  # skip empty lines
                continue
            label = int(parts[0])
            text = ' '.join(parts[1:])
            text_data.append({'label': label, 'text': text})
    
    # Create pandas DataFrame where each item is a dictionary (key & value)
    text_df = pd.DataFrame(text_data)
    labels = text_df['label'].values
    texts = text_df['text'].values
    
    vector_data = []
    with open(vector_file, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split()
            if not parts:  # skip empty lines
                continue
            label = int(parts[0])
            features = {}
            for item in parts[1:]:
                idx, val = item.split(':')
                # '3:1' to idx = '3', val = '1'
                features[int(idx)] = float(val)
            vector_data.append(features)
    
    # Convert dictionaries into pandas DataFrame
    vector_df = pd.DataFrame(vector_data).fillna(0)
    
    return texts, labels, vector_df

def preprocess_text(text_series):
    """      
        This reduces the dimensionality of text data and performance of downstram tasks like text classificaiton or information retrieval
    """   
    cleaned_texts = []
    tokenized_texts = []
    
    for text in text_series:
        tokens = word_tokenize(text)
        # Store the tokenized version for Word2Vec
        tokenized_texts.append(tokens)
        
        # Join tokens back into a string
        cleaned_text = ' '.join(tokens)
        cleaned_texts.append(cleaned_text)
    
    # cleaned_texts for text classification and feature extration techiques like TF-IDF
    # tokenized_text for token list used lated on word embeddings (Word2Vec) or RNN
    return cleaned_texts, tokenized_texts

def visualize_data(labels, texts, cleaned_texts):
    """
        Visualize data distributions and characteristics
    """
    # Distribution of classes with -1 and 1
    plt.figure(figsize=(8, 6)) #(width, height)
    sns.countplot(x=labels)
    plt.title('Distribution of Ad Classes')
    plt.xlabel('Class (-1: Not Accepted, 1: Accepted)')
    plt.ylabel('Count')
    plt.savefig('visualization/distributions.png')  
    plt.close()
    
    all_words = [word for text in cleaned_texts for word in text.split()]
    word_freq = pd.Series(all_words).value_counts()
    
    plt.figure(figsize=(12, 6))
    word_freq[:20].plot(kind='bar')
    plt.title('Top 20 Most Common Words')
    plt.xlabel('Words')
    plt.ylabel('Frequency')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig('visualization/word_frequency.png')
    plt.close()
    
    # Compare word frequencies between classes
    accepted_words = [word for i, text in enumerate(cleaned_texts)
                      for word in text.split() if labels[i] == 1]
    rejected_words = [word for i, text in enumerate(cleaned_texts)
                      for word in text.split() if labels[i] == -1]
    
    accepted_freq = pd.Series(accepted_words).value_counts()[:15]
    rejected_freq = pd.Series(rejected_words).value_counts()[:15]
    
    plt.figure(figsize=(16, 6))

    plt.subplot(1, 2, 1)
    accepted_freq.plot(kind='bar')
    plt.title('Top Words in Accepted Ads')
    plt.xlabel('Words')
    plt.ylabel('Frequency')
    plt.xticks(rotation=45, ha='right')
    
    plt.subplot(1, 2, 2)
    rejected_freq.plot(kind='bar')
    plt.title('Top Words in Rejected Ads')
    plt.xlabel('Words')
    plt.ylabel('Frequency')
    plt.xticks(rotation=45, ha='right')
    
    plt.tight_layout()
    plt.savefig('visualization/class_word_frequency.png')
    plt.close()

# Build a deep CNN model with at least 10 layers
def build_cnn_model(vocab_size, embedding_dim, embedding_matrix=None):
    """
    It compiles the model with binary cross-entropy loss (How wrong are the predictions "Yes, No" Decisions) and the Adam optimizer (gradually improved by minimizing the errors).
    
    Learn patterns from text data to decide between two classes (positive vs negative).
    Word embedding to turn words into numbers ->
    Goes to several layers to detect important features ->
    use normalization and dropout to learn better + avoid overfitting
    """
    model = Sequential()
    
    # Layer 1: Embedding layer
    if embedding_matrix is not None:
        model.add(Embedding(
            vocab_size, 
            embedding_dim, 
            weights=[embedding_matrix], 
            trainable=False
        ))
    else:
        model.add(Embedding(vocab_size, embedding_dim))
    
    # Layer 2: Spatial Dropout
    model.add(SpatialDropout1D(0.2))
    
    # Layers 3-4: Conv1D + BatchNorm
    model.add(Conv1D(64, 5, activation='relu'))
    model.add(BatchNormalization())
    
    # Layer 5: MaxPooling
    model.add(MaxPooling1D(3))
    
    # Layers 6-7: Conv1D + BatchNorm
    model.add(Conv1D(128, 5, activation='relu'))
    model.add(BatchNormalization())
    
    # Layer 8: MaxPooling
    model.add(MaxPooling1D(3))
    
    # Layers 9-10: Conv1D + BatchNorm
    model.add(Conv1D(256, 3, activation='relu'))
    model.add(BatchNormalization())
    
    # Layer 11: Global Max Pooling
    model.add(GlobalMaxPooling1D())
    
    # Layer 12: Dense + BatchNorm
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    
    # Layer 13: Dropout
    model.add(Dropout(0.3))
    
    # Layer 14: Dense output
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile model
    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(learning_rate=0.001),
                  metrics=['accuracy'])
    
    return model

# Build a LSTM model with at least 10 layers
def build_lstm_model(vocab_size, embedding_dim, embedding_matrix=None):
    """    
    This method creates a deep LSTM (Long Short-Term Memory) model that can learn patterns in sequences of words (like sentences), 
    especially useful for text classification. 
    It uses layers like LSTM (to capture sequence context), dropout (to prevent overfitting), 
    batch normalization (to stabilize learning), 
    and dense layers to make the final prediction between two classes (e.g., positive or negative).
    """
    model = Sequential()
    
    # Layer 1: Embedding layer
    if embedding_matrix is not None:
        model.add(Embedding(
            vocab_size, 
            embedding_dim, 
            weights=[embedding_matrix], 
            trainable=False
        ))
    else:
        model.add(Embedding(vocab_size, embedding_dim))
    
    # Layer 2: Spatial Dropout
    model.add(SpatialDropout1D(0.2))
    
    # Layer 3: LSTM
    model.add(LSTM(128, return_sequences=True))
    
    # Layer 4: Batch Normalization
    model.add(BatchNormalization())
    
    # Layer 5: Dropout
    model.add(Dropout(0.3))
    
    # Layer 6: LSTM
    model.add(LSTM(64, return_sequences=False))
    
    # Layer 7: Batch Normalization
    model.add(BatchNormalization())
    
    # Layer 8: Dropout
    model.add(Dropout(0.3))
    
    # Layer 9: Dense + Batch Normalization
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    
    # Layer 10: Dropout
    model.add(Dropout(0.3))
    
    # Layer 11: Dense output
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile model
    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(learning_rate=0.001),
                  metrics=['accuracy'])
    
    return model

# Build a BERT-based transfer learning model
def build_bert_model(max_length):
    """  
       This function creates a neural network for text classification.
        
       It uses a pre-trained BERT model as its base, adds several dense layers
       with batch normalization and dropout for further processing,
       and then compiles the entire model for binary classification.
    """
    # Import BERT model
    bert_model = TFBertModel.from_pretrained('bert-base-uncased')
    
    # Define input layers
    input_ids = Input(shape=(max_length,), dtype=tf.int32, name="input_ids")
    attention_mask = Input(shape=(max_length,), dtype=tf.int32, name="attention_mask")
    
    # Get BERT embeddings - use the model directly with inputs
    bert_outputs = bert_model({'input_ids': input_ids, 'attention_mask': attention_mask})
    
    # Use pooled output for classification
    pooled_output = bert_outputs.pooler_output
    
    # Add dropout
    x = Dropout(0.3)(pooled_output)
    
    # Hidden layers
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    
    x = Dense(64, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    
    # Output
    output = Dense(1, activation='sigmoid')(x)
    
    # Define model
    model = Model(inputs=[input_ids, attention_mask], outputs=output)
    
    # Freeze BERT layers
    bert_model.trainable = False
    
    # Compile model
    model.compile(
        loss='binary_crossentropy',
        optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=5e-5),
        metrics=['accuracy']
    )
    return model

def train_evaluate_model(model, X_train, y_train, X_test, y_test, batch_size=32, epochs=10, model_name="model"):
    """
    This method trains a neural network model using training data, validates it during training to avoid overfitting (with early stopping), 
    and then evaluates its performance on test data using accuracy, classification report, confusion matrix, and ROC curve. 
    It also saves training history and evaluation plots (accuracy/loss, confusion matrix, and ROC curve) to files for later review.
    """
    # Early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    
    # Train model
    history = model.fit(
        X_train, y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_split=0.2,
        callbacks=[early_stopping],
        verbose=1
    )
    
    # Evaluate model
    y_pred_proba = model.predict(X_test)
    y_pred = (y_pred_proba > 0.5).astype(int)
    
    # Convert -1/1 back to 0/1 for evaluation if needed
    if -1 in y_test:
        y_test_01 = (y_test == 1).astype(int)
    else:
        y_test_01 = y_test
    
    # Metrics
    accuracy = accuracy_score(y_test_01, y_pred)
    report = classification_report(y_test_01, y_pred)
    conf_matrix = confusion_matrix(y_test_01, y_pred)
    
    # Plot training history
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title(f'{model_name} - Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(f'{model_name} - Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    
    plt.tight_layout()
    plt.savefig(f'visualization/{model_name}_training_history.png')
    plt.close()
    
    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Not Accepted', 'Accepted'],
                yticklabels=['Not Accepted', 'Accepted'])
    plt.title(f'{model_name} - Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.tight_layout()
    plt.savefig(f'visualization/{model_name}_confusion_matrix.png')
    plt.close()
    
    # Plot ROC curve
    fpr, tpr, _ = roc_curve(y_test_01, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'{model_name} - ROC Curve')
    plt.legend(loc="lower right")
    plt.savefig(f'visualization/{model_name}_roc_curve.png')
    plt.close()
    
    results = {
        'accuracy': accuracy,
        'report': report,
        'conf_matrix': conf_matrix,
        'roc_auc': roc_auc,
        'history': history
    }
    
    return results

def compare_models(model_results):
    """
    Compare multiple models and visualize their performance
    """
    # Extract metrics for comparison
    models = list(model_results.keys())
    accuracies = [model_results[model]['accuracy'] for model in models]
    roc_aucs = [model_results[model]['roc_auc'] for model in models]
    
    # Prepare data for bar plot
    metrics_df = pd.DataFrame({
        'Model': models + models,
        'Metric': ['Accuracy'] * len(models) + ['ROC AUC'] * len(models),
        'Value': accuracies + roc_aucs
    })
    
    # Create comparison bar plot
    plt.figure(figsize=(10, 6))
    sns.barplot(x='Model', y='Value', hue='Metric', data=metrics_df)
    plt.title('Model Performance Comparison')
    plt.xlabel('Model')
    plt.ylabel('Score')
    plt.ylim(0, 1)
    plt.legend(title='Metric')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('visualization/model_comparison.png')
    plt.close()
    
    # Create comparison table
    comparison_table = pd.DataFrame({
        'Model': models,
        'Accuracy': accuracies,
        'ROC AUC': roc_aucs
    })
    
    return comparison_table

if __name__ == "__main__":
    # File paths
    text_file = "farm-ads"
    vector_file = "farm-ads-vect"
    
    # Load data
    texts, labels, vector_df = load_farm_ads_data(text_file, vector_file)
    
    print(f"Loaded {len(texts)} text samples with labels: {np.unique(labels)}")
    
    # Preprocess text data
    cleaned_texts, tokenized_texts = preprocess_text(texts)
        
    visualize_data(labels, texts, cleaned_texts)
    
    # Convert labels from -1/1 to 0/1 for binary classification
    labels_01 = (labels == 1).astype(int)
    
    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(
        cleaned_texts, labels_01, test_size=0.2, random_state=42, stratify=labels_01
    )
    
    # Keep original labels for model evaluation
    _, _, y_train_orig, y_test_orig = train_test_split(
        cleaned_texts, labels, test_size=0.2, random_state=42, stratify=labels
    )
    
    print(f"Training set size: {len(X_train)}, Test set size: {len(X_test)}")
    
    # Prepare data for models
    # Tokenize text for CNN and LSTM models
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(X_train)
    vocab_size = len(tokenizer.word_index) + 1 # +1 to ensure index 0 for padding in sequence models
    print(f"Vocabulary size: {vocab_size}")
    
    # Convert text to sequences
    X_train_seq = tokenizer.texts_to_sequences(X_train)
    X_test_seq = tokenizer.texts_to_sequences(X_test)
    
    # Pad sequences (makes sequences uniform in length for neural networks by adding 0 (padding) or cut off excess (truncation)
    max_length = 100  # Can adjust text length analysis
    X_train_pad = pad_sequences(X_train_seq, maxlen=max_length)
    X_test_pad = pad_sequences(X_test_seq, maxlen=max_length)
    
    # Create Word2Vec embeddings
    print("Training Word2Vec model...")
    w2v_model = Word2Vec(sentences=tokenized_texts, vector_size=100, window=5, min_count=1, workers=4)

    # Create embedding matrix for pre-trained embeddings
    embedding_dim = 100
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    for word, idx in tokenizer.word_index.items():
        if word in w2v_model.wv:
            embedding_matrix[idx] = w2v_model.wv[word]
    
    # Model 1: CNN with Word2Vec embeddings
    print("\nTraining CNN model with Word2Vec embeddings...")
    cnn_model = build_cnn_model(vocab_size, embedding_dim, embedding_matrix)
    cnn_results = train_evaluate_model(
        cnn_model, X_train_pad, y_train, X_test_pad, y_test_orig, 
        batch_size=32, epochs=15, model_name="CNN_Word2Vec"
    )

    # Model 2: LSTM with Word2Vec embeddings
    print("\nTraining LSTM model with Word2Vec embeddings...")
    lstm_model = build_lstm_model(vocab_size, embedding_dim, embedding_matrix)
    lstm_results = train_evaluate_model(
        lstm_model, X_train_pad, y_train, X_test_pad, y_test_orig, 
        batch_size=32, epochs=15, model_name="LSTM_Word2Vec"
    )
    
    # Model 3: BERT-based model (Transfer Learning)
    print("\nTraining BERT-based model (Transfer Learning)...")
    bert_max_length = 128
    bert_model = build_bert_model(bert_max_length)
    # Load tokenizer
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

    # Tokenize data
    train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=128)
    test_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=128)

    # Extract input_ids and attention_mask
    X_train_bert_ids = train_encodings["input_ids"]
    X_train_bert_masks = train_encodings["attention_mask"]
    X_test_bert_ids = test_encodings["input_ids"]
    X_test_bert_masks = test_encodings["attention_mask"]
    
    # Convert to numpy arrays
    X_train_bert_ids = np.array(X_train_bert_ids)
    X_train_bert_masks = np.array(X_train_bert_masks)
    X_test_bert_ids = np.array(X_test_bert_ids)
    X_test_bert_masks = np.array(X_test_bert_masks)
    y_train = np.array(y_train)

    bert_results = train_evaluate_model(
        bert_model, 
        (X_train_bert_ids, X_train_bert_masks), y_train,
        (X_test_bert_ids, X_test_bert_masks), y_test_orig,
        batch_size=16, epochs=5, model_name="BERT_Transfer"
    )
    
    # Compare all models
    all_model_results = {
        'CNN_Word2Vec': cnn_results,
        'LSTM_Word2Vec': lstm_results,
        'BERT_Transfer': bert_results
    }

    # Compare and visualize model performances
    comparison_table = compare_models(all_model_results)
    print("\nModel Comparison:")
    print(comparison_table)

    # Print classification reports for each model
    print("\nCNN Model Classification Report:")
    print(cnn_results['report'])

    print("\nLSTM Model Classification Report:")
    print(lstm_results['report'])

    print("\nBERT Model Classification Report:")
    print(bert_results['report'])

    # Saving Models
    # 1. Save Word2Vec (vectors and vocabulary)
    w2v_model.save("models/word2vec.model")
    np.save("models/embedding_matrix.npy", embedding_matrix)
    # 2. Save CNN (architecture 'layers', weights, optimizer state)
    cnn_model.save("models/cnn_word2vec.keras")
    # 3. Save LSTM 
    lstm_model.save("models/lstm_word2vec.keras")
    # 4. Save Bert-based model
    bert_model.save("models/bert_transfer.keras")
    bert_model.save_weights("models/bert_transfer_weights.h5")
    
    

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/davidvuong/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/davidvuong/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Loaded 4143 text samples with labels: [-1  1]
Training set size: 3314, Test set size: 829
Vocabulary size: 39293
Training Word2Vec model...

Training CNN model with Word2Vec embeddings...
Epoch 1/15
83/83 [==============================] - 2s 13ms/step - loss: 0.5310 - accuracy: 0.7458 - val_loss: 0.3699 - val_accuracy: 0.8341
Epoch 2/15
83/83 [==============================] - 1s 12ms/step - loss: 0.3987 - accuracy: 0.8159 - val_loss: 0.3383 - val_accuracy: 0.8416
Epoch 3/15
83/83 [==============================] - 1s 14ms/step - loss: 0.3321 - accuracy: 0.8578 - val_loss: 0.2839 - val_accuracy: 0.8884
Epoch 4/15
83/83 [==============================] - 1s 14ms/step - loss: 0.3037 - accuracy: 0.8710 - val_loss: 0.4117 - val_accuracy: 0.8492
Epoch 5/15
83/83 [==============================] - 1s 14ms/step - loss: 0.2771 - accuracy: 0.8846 - val_loss: 0.2993 - val_accuracy: 0.8763
Epoch 6/15
83/83 [==============================] - 1s 13ms/step - loss: 0.2531 - accuracy: 0.9012 - val_lo

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Epoch 1/5
166/166 [==============================] - 208s 1s/step - loss: 0.9722 - accuracy: 0.5157 - val_loss: 0.6885 - val_accuracy: 0.5113
Epoch 2/5
166/166 [==============================] - 202s 1s/step - loss: 0.9351 - accuracy: 0.5285 - val_loss: 0.6712 - val_accuracy: 0.5988
Epoch 3/5
26/26 [==============================] - 40s 2s/step


/opt/anaconda3/envs/ml_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/ml_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/ml_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Model Comparison:
           Model  Accuracy   ROC AUC
0   CNN_Word2Vec  0.851628  0.928216
1  LSTM_Word2Vec  0.845597  0.928034
2  BERT_Transfer  0.533172  0.673360

CNN Model Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.77      0.83       387
           1       0.82      0.93      0.87       442

    accuracy                           0.85       829
   macro avg       0.86      0.85      0.85       829
weighted avg       0.86      0.85      0.85       829


LSTM Model Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.79      0.83       387
           1       0.83      0.90      0.86       442

    accuracy                           0.85       829
   macro avg       0.85      0.84      0.84       829
weighted avg       0.85      0.85      0.84       829


BERT Model Classification Report:
              precision    recall  f1-score   support

           0       0

/opt/anaconda3/envs/ml_env/lib/python3.11/site-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)
